In [6]:
import os
import sys
import json
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel

import os
import json
from multiprocessing import Pool
from torch.utils.data import Dataset

sys.path.append('/root/tuning_space/Components/')
from Static import prompt_dict, si

raw_data_path='/root/autodl-tmp/weights/smile/data'
aim_path='/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl'
model_path = '/root/autodl-tmp/weights/chatglm3-6b'

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)



In [ ]:
with open('/root/autodl-tmp/dataset/OESD-GG-zh_cn-1/single_query.jsonl') as file:
    for line in file:
        print(line)
        

In [4]:
class instruction_dataset(Dataset):
    def __init__(self, data_path, tokenizer, truncate_length, max_query_length, query_key, answer_key, max_sample=None, num_workers=12):
        super().__init__()
        self.tokenizer = tokenizer
        self.truncate_length = truncate_length
        self.max_query_length = max_query_length
        self.query_key = query_key
        self.answer_key = answer_key
        self.examples = []

        # 使用多进程读取和处理数据
        with open(data_path, 'r') as file:
            lines = file.readlines()
            if max_sample:
                lines = lines[:max_sample]

        with Pool(num_workers) as p:
            self.examples = p.map(self.process_line, lines)

    def process_line(self, line):
        sample=json.loads(line)
        max_answer_length = self.truncate_length - self.max_query_length - 3

        # 判断query的长度
        query = sample[self.query_key]
        query_ids = self.tokenizer.encode(query, add_special_tokens=False)
        if len(query_ids) > self.max_query_length:
            query_ids = query_ids[:self.max_query_length]

        # 判断answer的长度
        answer = sample[self.answer_key]
        answer_ids = self.tokenizer.encode(answer, add_special_tokens=False)
        if len(answer) > max_answer_length:
            answer_ids = answer_ids[:max_answer_length]

        # 合并
        input_ids = query_ids + [si['[gMASK]']] + [si['sop']] + answer_ids + [si['eop']]
        pre_context_length = input_ids.index(si['sop'])
        end_answer_index = input_ids.index(si['eop'])

        # padding
        padding_length=self.truncate_length-len(input_ids)
        input_ids+=padding_length*[self.tokenizer.pad_token_id]

        # 制作labels；其中query部分，pad部分均不参与loss的计算 # 因为需要整体向左移动，所以要少填充一个
        labels = [-100] * (pre_context_length+1) + input_ids[pre_context_length+1: end_answer_index+1]
        labels = labels + [-100] * (self.truncate_length-len(labels))

        # 制作attention_mask
        eop_position = input_ids.index(si['eop'])+1
        attention_mask = [True]*eop_position
        attention_mask += [False]*(self.truncate_length-len(attention_mask))

        return {
            'query': query,
            'answer': answer,
            'input_ids': input_ids,
            'labels': labels,
            'attention_mask': attention_mask,
        }

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return self.examples[item]

In [5]:
temp=instruction_dataset(data_path=aim_path, tokenizer=tokenizer, truncate_length=192, max_query_length=64, query_key='User', answer_key='Assisstant', max_sample=5, num_workers=1)

KeyError: 'User'

In [ ]:
sample=None
for item in temp:
    sample=item
    break

In [ ]:
for item in sample.values():
    print(len(item))

In [2]:
input_ids=sample['input_ids']
labels=sample['labels']
attention_mask=sample['attention_mask']

NameError: name 'sample' is not defined

In [7]:
tokenizer.decode(input_ids)

'我最近遇到的苦恼是，我周围的人经常炫耀他们的成就和成功，而这让我感到非常不适。我发现自己的好胜心太强，总是希望自己能够取得更多的成就，但是当看到别人取得成功时，我就会感到自卑和沮丧。这种情况经常发生在工作场合或社交场合，不论是同事还是[gMASK]sop 我完全理解你的苦恼和困惑。看到他人的成功会引发对自己的不满和自卑感，这是很正常的情绪反应。社交和工作场合中的竞争氛围很容易让我们陷入比较和对抗的心态。但是，请放心，你并不孤单，许多人都经历过类似的感受。\n\n首先，我想告诉你，你的成就和价值并不仅仅是与他人的比较有关。每个人的经历和成就都是不同的，我们每个人都有自己的优点和才能。成功是多维度的，不仅仅局限于职位晋升或奖项。你可以尝试关注自己在工作和生活中的成长，寻找那些让你感到满足和自豪的事物。\n\n此外，记住与他人分享快乐和成功eop'

In [8]:
tokenizer.decode(labels)

'我完全理解你的苦恼和困惑。看到他人的成功会引发对自己的不满和自卑感，这是很正常的情绪反应。社交和工作场合中的竞争氛围很容易让我们陷入比较和对抗的心态。但是，请放心，你并不孤单，许多人都经历过类似的感受。\n\n首先，我想告诉你，你的成就和价值并不仅仅是与他人的比较有关。每个人的经历和成就都是不同的，我们每个人都有自己的优点和才能。成功是多维度的，不仅仅局限于职位晋升或奖项。你可以尝试关注自己在工作和生活中的成长，寻找那些让你感到满足和自豪的事物。\n\n此外，记住与他人分享快乐和成功eop'

In [15]:
tokenizer.bos_token

Using bos_token, but it is not set yet.


In [21]:
message={'role':'user', 'content':'你好'}
temp=tokenizer.build_single_message(
                    message['role'], '', message['content']
                )
tokenizer.decode(temp)

'<|user|> \n 你好'

In [23]:
message={'role':'assistant', 'content':'你好'}
temp=tokenizer.build_single_message(
                    message['role'], '', message['content']
                )
tokenizer.decode(temp)

'<|assistant|> \n 你好'

In [7]:
def process_batch(
        batch,
        tokenizer,
        max_input_length: int,
        max_output_length: int,
):
    batched_tools = batch.get('tools', None)
    batched_conv = batch['conversations']
    batched_input_ids = []
    batched_labels = []

    if batched_tools is None:
        batched_tools = [None] * len(batched_conv)

    for tools, conv in zip(batched_tools, batched_conv):
        input_ids, loss_masks = [
            tokenizer.get_command('[gMASK]'),
            tokenizer.get_command('sop'),
        ], [False, False]

        if tools is not None:
            raise NotImplementedError()

        for message in conv:
            if message['role'] in ('system', 'user'):
                loss_mask_val = False
            else:
                loss_mask_val = True

            if message['role'] == 'tool':
                raise NotImplementedError()
            else:
                new_input_ids = tokenizer.build_single_message(
                    message['role'], '', message['content']
                )
                new_loss_masks = [loss_mask_val] * len(new_input_ids)

            input_ids += new_input_ids
            loss_masks += new_loss_masks

        input_ids.append(tokenizer.eos_token_id)
        loss_masks = [False, *loss_masks]
        labels = []
        for input_id, mask in zip(input_ids, loss_masks):
            if mask:
                labels.append(input_id)
            else:
                labels.append(-100)
        max_length = max_input_length + max_output_length + 1
        batched_input_ids.append(input_ids[:max_length])
        batched_labels.append(labels[:max_length])
    return {'input_ids': batched_input_ids, 'labels': batched_labels}

In [8]:
# 模拟的batch数据
batch = {
    'conversations': [
        [
            {'role': 'system', 'content': 'I am glm'},
            {'role': 'user', 'content': 'Hello'},
            {'role': 'assistant', 'content': 'Hello! How can I help you?'},
            {'role': 'user', 'content': 'I need information on OpenAI.'},
            {'role': 'assistant', 'content': 'OpenAI provides AI research and deployment.'},
            {'role': 'user', 'content': 'That is great'},
            {'role': 'assistant', 'content': 'have a good day!'},
        ]
    ]
}

# 创建一个简化的tokenizer实例

# 调用process_batch函数
processed_batch = process_batch(
    batch=batch,
    tokenizer=tokenizer,
    max_input_length=50,
    max_output_length=50
)

print(processed_batch)

{'input_ids': [[64790, 64792, 64794, 30910, 13, 307, 674, 1175, 30924, 64795, 30910, 13, 13755, 64796, 30910, 13, 13755, 30992, 1072, 457, 307, 833, 344, 30987, 64795, 30910, 13, 307, 720, 1097, 331, 4949, 23833, 30930, 64796, 30910, 13, 4949, 23833, 2736, 11265, 1769, 293, 16400, 30930, 64795, 30910, 13, 1313, 323, 941, 64796, 30910, 13, 431, 260, 878, 1023, 30992, 2]], 'labels': [[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 13, 13755, 30992, 1072, 457, 307, 833, 344, 30987, 64795, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 30910, 13, 4949, 23833, 2736, 11265, 1769, 293, 16400, 30930, 64795, -100, -100, -100, -100, -100, -100, 30910, 13, 431, 260, 878, 1023, 30992, 2]]}


In [9]:
tokenizer.decode(processed_batch['input_ids'][0])

'[gMASK]sop<|system|> \n I am glm<|user|> \n Hello<|assistant|> \n Hello! How can I help you?<|user|> \n I need information on OpenAI.<|assistant|> \n OpenAI provides AI research and deployment.<|user|> \n That is great<|assistant|> \n have a good day!'

In [10]:
tokenizer.decode(processed_batch['labels'][0])

'\n Hello! How can I help you?<|user|> \n OpenAI provides AI research and deployment.<|user|> \n have a good day!'

In [11]:
tokenizer.decode(64796), tokenizer.decode(64794)

('<|assistant|>', '<|system|>')

In [12]:
'\n 看到你的提问感觉你很焦虑，这个状态在高中高压下很容易出现。我想说的是，我看到了你的决心。这点是很多人没有的！高考最重要的不是知识是心态。是必胜的心态！什么放松吧缓缓吧，都是站着说话不腰疼，保送的又不是我，我能放松什么？！我有我的目标，我怎么可能放弃！有目标就好办，计划！缺个计划，缺个时间合理配置的复习计划。<|user|>\n 首先，你要明确自己的目标，既然你想考本科，那就要为此做好准备。然后，你需要制定一个合理的复习计划，根据自己的情况来安排每天的学习时间和内容。这样可以帮助你更好地掌控整个复习过程，减少焦虑感。<|user|>\n 当然可以！你可以从高一开始，试试题海战术。每天多做一些题目，这样能够提高你的学习效率。同时，对于英语这门科目，多听多背是很重要的，数理化方面，可以做一些经典的题目，特别是那些类型经常考到的题目，多次反复做题。<|user|>\n 不要自我怀疑，这只会增加你的心理负担。如果遇到难题，你可以大胆去问老师，他们就是为了解答你的问题而存在的。不要担心别人的期望，你应该相信自己的潜力，只要你拼命学习，一定会有所收获的。<|user|>\n 很高兴能够帮到你！记住要保持信心，坚持努力，我相信你一定能够取得优异的成绩。加油！'

'\n 看到你的提问感觉你很焦虑，这个状态在高中高压下很容易出现。我想说的是，我看到了你的决心。这点是很多人没有的！高考最重要的不是知识是心态。是必胜的心态！什么放松吧缓缓吧，都是站着说话不腰疼，保送的又不是我，我能放松什么？！我有我的目标，我怎么可能放弃！有目标就好办，计划！缺个计划，缺个时间合理配置的复习计划。<|user|>\n 首先，你要明确自己的目标，既然你想考本科，那就要为此做好准备。然后，你需要制定一个合理的复习计划，根据自己的情况来安排每天的学习时间和内容。这样可以帮助你更好地掌控整个复习过程，减少焦虑感。<|user|>\n 当然可以！你可以从高一开始，试试题海战术。每天多做一些题目，这样能够提高你的学习效率。同时，对于英语这门科目，多听多背是很重要的，数理化方面，可以做一些经典的题目，特别是那些类型经常考到的题目，多次反复做题。<|user|>\n 不要自我怀疑，这只会增加你的心理负担。如果遇到难题，你可以大胆去问老师，他们就是为了解答你的问题而存在的。不要担心别人的期望，你应该相信自己的潜力，只要你拼命学习，一定会有所收获的。<|user|>\n 很高兴能够帮到你！记住要保持信心，坚持努力，我相信你一定能够取得优异的成绩。加油！'